# Pre-Processing of data

In [20]:
#importing the required libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
data=pd.read_csv(r"C:\Users\asus\Documents\Projects\Question_Similarity_Pairs\Input\train.csv")#.sample(6000)

In [10]:
data.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [11]:
data.dropna(axis=0,subset=['question1','question2'],how='any',inplace=True)

In [12]:
ques1_list=list(set(data['question1'].tolist()))
ques2_list=list(set(data['question2'].tolist()))
print("Unique question in question_1:",len(ques1_list))
print("Unique question in question_2:", len(ques2_list))
fin_list=ques1_list+ques2_list
print("Total % of Unique questions that we have in this corpus is :",len(fin_list)/(2*data.id.count()))

Unique question in question_1: 290455
Unique question in question_2: 299173
Total % of Unique questions that we have in this corpus is : 0.7292195890543104


In [21]:
def pre_process(x):
    #Normalization
    x=str(x).lower()
    #Removing Contraction
    x=x.replace("000,000","m").replace("000","k").replace("won't","will not").replace("can't","cannot").replace("shouldn't","should not")\
        .replace("didn't","did not").replace("doesn't","does not").replace("couldn't","could not").replace("ll","will").replace("%","percent")\
            .replace("$","dollar").replace("he's","he is").replace("he's","he is").replace("wouldn't","would not")
    #Removeing Unicode Characters
    x=re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "",x)
    #Removing Stopwords
    text=[word for word in x.split() if word  not in (stop)]
    #Stemming
    stemmer=PorterStemmer()
    text=" ".join([stemmer.stem(word) for word in text])
    #removing HTML tags
    bs4=BeautifulSoup(text)
    text=bs4.get_text()
    text=text.strip()
    return text


    
    


In [22]:
def cleaning(val):
    return data[val].apply(pre_process)

data['question1']=cleaning('question1')
data['question2']=cleaning('question2')

    

In [24]:
data.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,step step guid invest share market india,step step guid invest share market,0
1,1,3,4,stori kohinoor kohinoor diamond,would happen indian govern stole kohinoor kohi...,0
2,2,5,6,increa speed internet connect use vpn,internet speed increa hack dn,0
3,3,7,8,mentawiwilli lone solv,find remaind math2324math divid 2423,0
4,4,9,10,one dissolv water quikli sugar salt methan car...,fish would surviv salt water,0
5,5,11,12,astrolog capricorn sun cap moon cap risingwhat...,im tripl capricorn sun moon ascend capricorn say,1
6,6,13,14,buy tiago,keep childern activ far phone video game,0
7,7,15,16,good geologist,great geologist,1
8,8,17,18,use instead,use instead,0
9,9,19,20,motorola compani hack charter motorowiwilla dc...,hack motorola dcx3400 free internet,0


In [26]:
data.to_csv(r"C:\Users\asus\Documents\Projects\Question_Similarity_Pairs\Input\preprocessed_data.csv")